# Creating and Valuing a CDS Contract

Replicating Markit Pricing on 20 Aug 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 21:20 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating a CDS Contract

In [3]:
value_dt = Date(20, 8, 2020)
effective_dt = Date(21, 8, 2020)
maturity_dt = Date(20, 6, 2025)
running_cpn = 0.05
notional = ONE_MILLION
long_protection = True

In [4]:
cds_contract = CDS(effective_dt, maturity_dt, running_cpn, notional, long_protection)

## Build Ibor Curve

In [5]:
settle_dt = Date(24, 8, 2020)

In [6]:
dc_type = DayCountTypes.ACT_360
depo1 = IborDeposit(settle_dt, "1M", 0.001709, dc_type)
depo2 = IborDeposit(settle_dt, "2M", 0.002123, dc_type)
depo3 = IborDeposit(settle_dt, "3M", 0.002469, dc_type)
depo4 = IborDeposit(settle_dt, "6M", 0.003045, dc_type)
depo5 = IborDeposit(settle_dt, "12M", 0.004449, dc_type)
depos = [depo1,depo2,depo3,depo4,depo5]

swap_type = SwapTypes.PAY
dc_type = DayCountTypes.THIRTY_E_360_ISDA
fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap1 = IborSwap(settle_dt,"2Y", swap_type, 0.002155,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"3Y", swap_type, 0.002305,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"4Y", swap_type, 0.002665,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"5Y", swap_type, 0.003290,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"6Y", swap_type, 0.004025,fixed_freq,dc_type)
swap6 = IborSwap(settle_dt,"7Y", swap_type, 0.004725,fixed_freq,dc_type)
swap7 = IborSwap(settle_dt,"8Y", swap_type, 0.005430,fixed_freq,dc_type)
swap8 = IborSwap(settle_dt,"9Y", swap_type, 0.006075,fixed_freq,dc_type)
swap9 = IborSwap(settle_dt,"10Y", swap_type, 0.006640,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

# Build a CDS Curve

In [7]:
cdsSpread = 0.01

In [8]:
cds1 = CDS(settle_dt, "6M", cdsSpread)
cds2 = CDS(settle_dt, "1Y", cdsSpread)
cds3 = CDS(settle_dt, "2Y", cdsSpread)
cds4 = CDS(settle_dt, "3Y", cdsSpread)
cds5 = CDS(settle_dt, "4Y", cdsSpread)
cds6 = CDS(settle_dt, "5Y", cdsSpread)
cds7 = CDS(settle_dt, "7Y", cdsSpread)
cds8 = CDS(settle_dt, "10Y", cdsSpread)

In [9]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [10]:
recovery_rate = 0.40

In [11]:
issuer_curve = CDSCurve(value_dt, cdss, libor_curve, recovery_rate)

In [12]:
print(issuer_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.5808219,  0.9901922
 1.0849315,  0.9817979
 2.0849315,  0.9986272
 3.0849315,  0.9818007
 4.0876712,  0.9651915
 5.0876712,  0.9490150
 7.0876712,  0.9174925
10.0904110,  0.8721194


In [13]:
cds_contract.print_payments(value_dt, issuer_curve) 

PAYMENT_dt      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
    21-SEP-2020   0.252778     12638.89     0.999848     0.998513     12618.18
    21-DEC-2020   0.252778     12638.89     0.999079     0.994298     12555.25
    22-MAR-2021   0.252778     12638.89     0.998005     0.990101     12488.80
    21-JUN-2021   0.252778     12638.89     0.996528     0.985941     12417.93
    20-SEP-2021   0.252778     12638.89     0.995505     0.981798     12353.06
    20-DEC-2021   0.252778     12638.89     0.995554     0.985967     12406.12
    21-MAR-2022   0.252778     12638.89     0.995603     0.990154     12459.42
    20-JUN-2022   0.252778     12638.89     0.995652     0.994358     12512.94
    20-SEP-2022   0.255556     12777.78     0.995495     0.998627     12702.75
    20-DEC-2022   0.252778     12638.89     0.994848     0.994405     12503.43
    20-MAR-2023   0.250000     12500.00     0.994209     0.990247     12306.41
    20-JUN-2023   0.255556     12777.78     0.993556     

# Valuation

In [14]:
spd = cds_contract.par_spread(settle_dt, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD   99.08198 bp


In [15]:
cds_contract.value_fast_approx(settle_dt, 0.004, 0.01, 0.40, 0.40)

(-195533.6454106767, -187200.31207734335, 533.6950180429558, 44.32197353540687)

In [16]:
v = cds_contract.value(settle_dt, issuer_curve, recovery_rate)

In [17]:
dirty_pv = v['dirty_pv'] 
clean_pv = v['clean_pv']

In [18]:
print("DIRTY VALUE  %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE    -200072.73
CLEAN VALUE   -191739.40


In [19]:
cleanp = cds_contract.clean_price(settle_dt, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   119.173940


In [20]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 60.0


In [21]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8333.333333333334


In [22]:
prot_pv = cds_contract.prot_leg_pv(settle_dt, issuer_curve, recovery_rate)
print("prot_PV", prot_pv)

prot_PV 47386.047104632395


In [23]:
premPV = cds_contract.premium_leg_pv(settle_dt, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 247458.7795293185


In [24]:
cds_contract.risky_pv01(settle_dt, issuer_curve)

{'dirty_rpv01': 4.9491755905863695, 'clean_rpv01': 4.7825089239197025}

## Risk Measures

In [25]:
cds_contract.credit_dv01(settle_dt, issuer_curve, recovery_rate)

507.9938544575998

In [26]:
cds_contract.interest_dv01(settle_dt, issuer_curve, recovery_rate)

48.10582343424903

Copyright (c) 2020 Dominic O'Kane